In [31]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import tensorflow as tf
from transformers import DistilBertTokenizerFast, \
    DistilBertTokenizer, TFDistilBertModel, \
    TFDistilBertForSequenceClassification, DistilBertConfig
from tqdm import tqdm

np.set_printoptions(precision=4, suppress=True)

In [2]:
tf.config.list_physical_devices()

2023-02-20 13:05:42.332315: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-02-20 13:05:42.332345: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-02-20 13:05:42.332365: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ecs-irukhovich): /proc/driver/nvidia/version does not exist


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [3]:
data_folder = "../data/"
train_path = "train.csv"
test_path = "test.csv"

random_state = 12345

In [4]:
df_test = pd.read_csv(data_folder + test_path, index_col=0)
df_test

,Title,Description
ID,,
0,Fears for T N pension after talks,Unions representing workers at Turner Newall...
1,The Race is On: Second Private Team Sets Launc...,"SPACE.com - TORONTO, Canada -- A second\team o..."
2,Ky. Company Wins Grant to Study Peptides (AP),AP - A company founded by a chemistry research...
3,Prediction Unit Helps Forecast Wildfires (AP),AP - It's barely dawn when Mike Fitzpatrick st...
4,Calif. Aims to Limit Farm-Related Smog (AP),AP - Southern California's smog-fighting agenc...
...,...,...
7595,Around the world,Ukrainian presidential candidate Viktor Yushch...
7596,Void is filled with Clement,With the supply of attractive pitching options...
7597,Martinez leaves bitter,Like Roger Clemens did almost exactly eight ye...


In [5]:
df_train = pd.read_csv(data_folder + train_path)
df_train

,Class Index,Title,Description
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli..."
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...
4,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco..."
...,...,...,...
119995,1,Pakistan's Musharraf Says Won't Quit as Army C...,KARACHI (Reuters) - Pakistani President Perve...
119996,2,Renteria signing a top-shelf deal,Red Sox general manager Theo Epstein acknowled...
119997,2,Saban not going to Dolphins yet,The Miami Dolphins will put their courtship of...
119998,2,Today's NFL games,PITTSBURGH at NY GIANTS Time: 1:30 p.m. Line: ...


In [6]:
# World (politics)
df_train.loc[df_train["Class Index"] == 1]

,Class Index,Title,Description
492,1,Venezuelans Vote Early in Referendum on Chavez...,Reuters - Venezuelans turned out early\and in ...
493,1,S.Koreans Clash with Police on Iraq Troop Disp...,Reuters - South Korean police used water canno...
494,1,Palestinians in Israeli Jails Start Hunger Str...,Reuters - Thousands of Palestinian\prisoners i...
495,1,Seven Georgian soldiers wounded as South Osset...,AFP - Sporadic gunfire and shelling took place...
496,1,Rwandan Troops Arrive in Darfur (AP),AP - Dozens of Rwandan soldiers flew into Suda...
...,...,...,...
119990,1,Barack Obama Gets #36;1.9 Million Book Deal (AP),"AP - U.S. Sen.-elect Barack Obama, whose 1995 ..."
119992,1,Iraqis Face Winter Shivering by Candlelight,BAGHDAD (Reuters) - As if the daily struggle ...
119993,1,AU Says Sudan Begins Troop Withdrawal from Darfur,ABUJA (Reuters) - The African Union said on S...
119994,1,Syria Redeploys Some Security Forces in Lebanon,"BEIRUT (Reuters) - Syria, under intense press..."


In [7]:
# Sport
df_train.loc[df_train["Class Index"] == 2]

,Class Index,Title,Description
448,2,"Phelps, Thorpe Advance in 200 Freestyle (AP)",AP - Michael Phelps took care of qualifying fo...
449,2,Reds Knock Padres Out of Wild-Card Lead (AP),AP - Wily Mo Pena homered twice and drove in f...
450,2,"Dreaming done, NBA stars awaken to harsh Olymp...",AFP - National Basketball Association players ...
451,2,"Indians Beat Twins 7-1, Nearing AL Lead (AP)",AP - The Cleveland Indians pulled within one g...
452,2,"Galaxy, Crew Play to 0-0 Tie (AP)",AP - Kevin Hartman made seven saves for Los An...
...,...,...,...
119991,2,Rauffer Beats Favorites to Win Downhill,"VAL GARDENA, Italy (Reuters) - Max Rauffer be..."
119996,2,Renteria signing a top-shelf deal,Red Sox general manager Theo Epstein acknowled...
119997,2,Saban not going to Dolphins yet,The Miami Dolphins will put their courtship of...
119998,2,Today's NFL games,PITTSBURGH at NY GIANTS Time: 1:30 p.m. Line: ...


In [8]:
# Business (Stocks)
df_train.loc[df_train["Class Index"] == 3]

,Class Index,Title,Description
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli..."
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...
4,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco..."
...,...,...,...
119977,3,"Despite Mega-Deals, Small Mergers Rule",Philadelphia (Reuters) - Despite the euphoria...
119978,3,Wal-Mart Dec. Sales Still Seen Up 1-3 Pct,CHICAGO (Reuters) - Wal-Mart Stores Inc. &lt;...
119979,3,Sabotage Stops Iraq's North Oil Exports,BAGHDAD (Reuters) - Saboteurs blew up Iraq's ...
119980,3,Corporate Cost-Cutters Miss Out,CHICAGO (Reuters) - One of the mantras of glo...


In [9]:
# Sci-Tech (IT)
df_train.loc[df_train["Class Index"] == 4]

,Class Index,Title,Description
78,4,"'Madden,' 'ESPN' Football Score in Different W...",Reuters - Was absenteeism a little high\on Tue...
79,4,Group to Propose New High-Speed Wireless Forma...,Reuters - A group of technology companies\incl...
80,4,AOL to Sell Cheap PCs to Minorities and Senior...,Reuters - America Online on Thursday said it\p...
81,4,Companies Approve New High-Capacity Disc Forma...,Reuters - A group of consumer electronics\make...
82,4,Missing June Deals Slow to Return for Software...,Reuters - The mystery of what went wrong for t...
...,...,...,...
119940,4,"Naughty, nice are irrelevant if Santa can #39;...",You could spend all weekend exploring retail s...
119950,4,Digitized And Brought To Life,Digital technology is radically changing the 1...
119951,4,New Computer? Six Steps to Safer Surfing,To see the e-mail I get every day from readers...
119952,4,Video Files Present A Search Challenge,Indexing Web pages looks like child's play nex...


In [10]:
df_train["full"] = df_train["Title"] + " " + df_train["Description"]
df_train = df_train.loc[df_train["full"].str.count(' ') >= 10].copy()
df_test["full"] = df_test["Title"] + " " + df_test["Description"]
X = df_train["full"]
X_test = df_test["full"]
y = df_train["Class Index"] - 1
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.33, random_state=random_state)

print(X_train.shape, X_valid.shape, X_test.shape)

(80385,) (39593,) (7600,)


In [11]:
# Instantiate DistilBERT tokenizer...we use the Fast version to optimize runtime
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

In [12]:
# Define the maximum number of words to tokenize (DistilBERT can tokenize up to 512)
MAX_LENGTH = 72


# Define function to encode text data in batches
def batch_encode(tokenizer, texts, batch_size=256, max_length=MAX_LENGTH):
    """""""""
    A function that encodes a batch of texts and returns the texts'
    corresponding encodings and attention masks that are ready to be fed 
    into a pre-trained transformer model.
    
    Input:
        - tokenizer:   Tokenizer object from the PreTrainedTokenizer Class
        - texts:       List of strings where each string represents a text
        - batch_size:  Integer controlling number of texts in a batch
        - max_length:  Integer controlling max number of words to tokenize in a given text
    Output:
        - input_ids:       sequence of texts encoded as a tf.Tensor object
        - attention_mask:  the texts' attention mask encoded as a tf.Tensor object
    """""""""
    
    input_ids = []
    attention_mask = []
    
    for i in tqdm(range(0, len(texts), batch_size)):
        batch = texts[i:i+batch_size]
        inputs = tokenizer.batch_encode_plus(batch,
                                             max_length=max_length,
                                             padding='longest', #implements dynamic padding
                                             truncation=True,
                                             return_attention_mask=True,
                                             return_token_type_ids=False,
                                             )
        input_ids.extend(inputs['input_ids'])
        attention_mask.extend(inputs['attention_mask'])
    
    return tf.convert_to_tensor(input_ids), tf.convert_to_tensor(attention_mask)
    
    
# Encode X_train
X_train_ids, X_train_attention = batch_encode(tokenizer, X_train.to_list())

# Encode X_valid
X_valid_ids, X_valid_attention = batch_encode(tokenizer, X_valid.to_list())

# Encode X_test
X_test_ids, X_test_attention = batch_encode(tokenizer, X_test.to_list())


100%|██████████| 315/315 [01:05<00:00,  4.81it/s]
2023-02-20 13:06:51.682497: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
100%|██████████| 30/30 [00:06<00:00,  4.92it/s]


In [13]:
DISTILBERT_DROPOUT = 0.2
DISTILBERT_ATT_DROPOUT = 0.2
 
# Configure DistilBERT's initialization
# config = DistilBertConfig(dropout=DISTILBERT_DROPOUT, 
#                           attention_dropout=DISTILBERT_ATT_DROPOUT, 
#                           output_hidden_states=True)
config = DistilBertConfig(output_hidden_states=True, num_labels=4)
                          
# The bare, pre-trained DistilBERT transformer model outputting raw hidden-states 
# and without any specific head on top.
distilBERT = TFDistilBertModel.from_pretrained('distilbert-base-uncased',
                                               config=config)

# Make DistilBERT layers untrainable
for layer in distilBERT.layers:
    layer.trainable = False

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['activation_13', 'vocab_transform', 'vocab_layer_norm', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [14]:
MAX_LENGTH = 72
LEARNING_RATE = 5e-5
LAYER_DROPOUT = 0.2
RANDOM_STATE = random_state

def build_model(transformer, max_length=MAX_LENGTH):
    """
    Template for building a model off of the BERT or DistilBERT architecture
    for a binary classification task.
    
    Input:
      - transformer:  a base Hugging Face transformer model object (BERT or DistilBERT)
                      with no added classification head attached.
      - max_length:   integer controlling the maximum number of encoded tokens 
                      in a given sequence.
    
    Output:
      - model:        a compiled tf.keras.Model with added classification layers 
                      on top of the base pre-trained model architecture.
    """
    
    # Define weight initializer with a random seed to ensure reproducibility
    weight_initializer = tf.keras.initializers.GlorotNormal(seed=RANDOM_STATE) 
    
    # Define input layers
    input_ids_layer = tf.keras.layers.Input(shape=(max_length,), 
                                            name='input_ids', 
                                            dtype='int32')
    input_attention_layer = tf.keras.layers.Input(shape=(max_length,), 
                                                  name='input_attention', 
                                                  dtype='int32')
    
    # DistilBERT outputs a tuple where the first element at index 0
    # represents the hidden-state at the output of the model's last layer.
    # It is a tf.Tensor of shape (batch_size, sequence_length, hidden_size=768).
    last_hidden_state = transformer([input_ids_layer, input_attention_layer])[0]
    
    # We only care about DistilBERT's output for the [CLS] token, 
    # which is located at index 0 of every encoded sequence.  
    # Splicing out the [CLS] tokens gives us 2D data.
    cls_token = last_hidden_state[:, 0, :]
    
    X = tf.keras.layers.BatchNormalization()(cls_token)
    X = tf.keras.layers.Dense(192, activation='relu')(X)
    X = tf.keras.layers.Dropout(LAYER_DROPOUT)(X)
    
    # Define a single node that makes up the output layer (for binary classification)
    output = tf.keras.layers.Dense(4, 
                                   activation='softmax',
                                   kernel_initializer=weight_initializer,  
                                   kernel_constraint=None,
                                   bias_initializer='zeros'
                                   )(cls_token)
    
    # Define the model
    model = tf.keras.Model([input_ids_layer, input_attention_layer], output)
    
    # Compile the model
    model.compile(tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE), 
                  loss=tf.nn.softmax_cross_entropy_with_logits,
                  metrics=['accuracy'])
    
    return model

In [15]:
model = build_model(distilBERT)
model.summary()

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 72)]         0           []                               
                                                                                                  
 input_attention (InputLayer)   [(None, 72)]         0           []                               
                                                                                                  
 tf_distil_bert_model (TFDistil  TFBaseModelOutput(l  66362880   ['input_ids[0][0]',              
 BertModel)                     ast_hidden_state=(N               'input_attention[0][0]'] 

In [16]:
EPOCHS = 2
BATCH_SIZE = 4096
NUM_STEPS = len(X_train.index) // BATCH_SIZE

# Train the model
train_history1 = model.fit(
    x = [X_train_ids, X_train_attention],
    y = y_train.to_numpy(),
    epochs = EPOCHS,
    batch_size = BATCH_SIZE,
    steps_per_epoch = NUM_STEPS,
    validation_data = ([X_valid_ids, X_valid_attention], y_valid.to_numpy()),
    verbose=2
)

Epoch 1/2
19/19 - 727s - loss: 8.3446 - accuracy: 0.3143 - val_loss: 8.3500 - val_accuracy: 0.3348 - 727s/epoch - 38s/step
Epoch 2/2
19/19 - 782s - loss: 8.3367 - accuracy: 0.3190 - val_loss: 8.3433 - val_accuracy: 0.3422 - 782s/epoch - 41s/step


In [32]:
y_train_preds = model.predict([X_train_ids, X_train_attention])
y_valid_preds = model.predict([X_valid_ids, X_valid_attention])
y_test_preds = model.predict([X_test_ids, X_test_attention])
y_test_preds

238/238 [==============================] - 47s 196ms/step


array([[0.2278, 0.286 , 0.2303, 0.2559],
       [0.2311, 0.2393, 0.2332, 0.2964],
       [0.1341, 0.2309, 0.3785, 0.2566],
       ...,
       [0.0972, 0.2481, 0.3112, 0.3435],
       [0.1956, 0.3008, 0.2648, 0.2387],
       [0.1066, 0.2941, 0.3582, 0.2412]], dtype=float32)

In [36]:
print(f'Train f1 score: {f1_score(y_train, y_train_preds.argmax(axis=1), average="micro")}')
print(f'Valid f1 score: {f1_score(y_valid, y_valid_preds.argmax(axis=1), average="micro")}')

Train f1 score: 0.34095913416682216
Valid f1 score: 0.34220695577501076


In [37]:
submission_fname = "submission_v1.csv"

submission = pd.Series(y_test_preds.argmax(axis=1) + 1, name="Class Index")
submission.index.name = "ID"
submission.to_csv(data_folder + submission_fname)
submission

ID
0       2
1       4
2       3
3       4
4       3
       ..
7595    3
7596    2
7597    4
7598    2
7599    3
Name: Class Index, Length: 7600, dtype: int64